In [ ]:
# Configures the Environment (run it once)
include("configure.jl")

# Configuration of the Quadrotor

In [ ]:
include("Model_nd_Dynamics.jl")
include("Simulation.jl")
include("Trajectory.jl")
include("MPC.jl")
include("Close_loop.jl")

In [ ]:
## Model Configs
#    g::Float64 # gravity
#    m::Float64 # mass
#    ℓ::Float64 # inter-propeller distance
#    J::Float64 # inertia momment
#    R::Float64 # propeller radius
#    prop_min_h::Float64 #
#    ρ::Float64 # GE correction factor (https://doi.org/10.1109/ChiCC.2015.7260521)
#    umin::Vector{Float64} # Thrust inferior limit
#    umax::Vector{Float64} # Thrust superior limit

model = Model(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, [0.0, 0.0], [0.0, 0.0])
model.g = 9.81 # [m/s^2]
model.m = 1.0 # [kg]
model.ℓ = 0.3 # [meters]
model.J = 0.2 * model.m * model.ℓ * model.ℓ # [kg . m^2]
model.R = 0.15 # [meters]
model.prop_min_h = 0.1 # [meters]
model.ρ = 1.0 # dimensionless

#Thrust limits
model.umin = [0.0 * model.m * model.g; 0.0 * model.m * model.g]
model.umax = [2.0 * model.m * model.g; 2.0 * model.m * model.g]

## Simulation Configs
#
# Universe 1000Hz
# Controlador 100Hz

h_universe = 0.0001 # Time step (10 kHz)
h_controller = 0.004 # Time step (200 Hz)
Tfinal = 10.0 # Simulate from 0 - 10 seconds

simulation = Simulation(h_universe, h_controller, Tfinal)

## Trajectory configuration
#mutable struct Trajectory
#    Amp::Float64    # Sine amplitude - [meters]
#    freq::Float64   # Sine frequency - [Hz]
#end

landing_start_height = 3.0
stationary_height = model.prop_min_h + 0.06 # 6 centimeters above the ground
plat_amplitude = 0.1
plat_freq = 0.3

traj_params = Trajectory(plat_amplitude, plat_freq, landing_start_height, stationary_height)

## Linear State Space Configs ???
#struct LinearStateSpace
#    Nx::Int64          # State vector dimension
#    Nu::Int64          # Control vector dimension
#    A::Matrix{Float64} # State Matrix
#    B::Matrix{Float64} # Control Matrix
#    x_hover::Float64   # x value for which the linearization was made
#    u_hover::Float64   # u value for which the linearization was made
Nx = 6
Nu = 2

#    function LinearStateSpace(model, simulation, Nx, Nu, height)
state_space = LinearStateSpace(model, simulation, Nx, Nu, landing_start_height)

## Tunning Parameters Configs
# mutable struct MPCTunningParameters
#    Nh::Int64           # Predictive horizon
#    Nc::Int64           # Control horizon (Nc < Nh)
#    Q::Matrix{Float64}  # Stage State Cost Penalty
#    R::Matrix{Float64}  # Stage Control Cost Penalty
#    Qn::Matrix{Float64} # Terminal State Cost Penalty
#end
Nh = 10
Nc = 42 # For now is unused...

# Cost weights
Q = Array(10.0*I(Nx));
Q[2,2] = 1000.0
R = Array(.01*I(Nu));
Qn = Array(1000.0*I(Nx));

tunning_params = MPCTunningParameters(Nh, Nc, Q, R, Qn)

## MPC Matrices Configs

#mutable struct MPCMatrices
#    P::Matrix{Float64}
#    U::Matrix{Float64}
#    Θ::Matrix{Float64}
#    H::SparseMatrixCSC{Float64,Int64}
#    b::Array{Float64,1}
#    C::SparseMatrixCSC{Float64,Int64}
#    Y::Matrix{Float64}
#    D::Array{Float64,1}
#    lb::Array{Float64,1}
#    ub::Array{Float64,1}

# function MPCMatrices(model, state_space, tunning_params, trajectory)
#mpc_mats =  MPCMatrices(model, state_space, tunning_params, traj_params)"

#x_ref = [0.0; 1.0; 0; 0; 0; 0]
x0 = [1.0; traj_params.init_heigth; 0.0; 0; 0; 0]

#xhist1, uhist1 = closed_loop(x0, (t,x)->lqr_controller(t,x,K,x_ref), Nt);
#xhist2, uhist2 = closed_loop(x0, (t,x)->mpc_controller(t,x,x_ref), Nt);
xhist3, uhist3, plathist3, refyhist3, drefyhist3, land_time = closed_loop(
    x0,
    state_space,
    tunning_params, 
    traj_params,
    simulation,
    model,
    # mpc_controller(model, simulation, tunning_params, trajectory, t, x, xref)
    (t, x) -> mpc_controller(
        model,
        simulation,
        tunning_params,
        traj_params,
        t,
        x, 
        gen_ref(state_space, traj_params, tunning_params, t, simulation.h_controller) # verify if this works as expected!!
    )
);

In [ ]:
using Plots

thist = Array(range(0, simulation.h_universe * (simulation.Nt_universe - 1), step = simulation.h_universe));
#println(simulation.h_universe * (simulation.Nt_universe - 1))
plot(thist, xhist3[1,:], label="x MPC")

scatter!([land_time], [xhist3[1,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)
xlabel!("time")
savefig("Test01.svg")

#using  Images,  FileIO
#display(MIME("image/svg+xml"), read("Test01.svg", String))

In [ ]:
plot(thist, xhist3[4, :], label="x dot MPC")
scatter!([land_time], [xhist3[4,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

xlabel!("time")
savefig("Test02.svg")

In [ ]:
#plot(thist,xhist1[2,:], label=\"y LQR\")
plot(thist,xhist3[3,:], label="θ MPC")
scatter!([land_time], [xhist3[3,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)
xlabel!("time")

savefig("Test03.svg")

In [ ]:
plot(thist, xhist3[6, :], label="θ dot MPC")
scatter!([land_time], [xhist3[6,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

xlabel!("time")
savefig("Test04.svg")

In [ ]:
## Plot de interesse|
plot(thist,xhist3[2,:], label="y MPC")

#plot!(thist, plathist3 .+ traj_params.stat_height, label=\"traj\")
plot!(thist, refyhist3, label="traj")
plot!(thist, plathist3, label="plat")

xlabel!("time")
scatter!([land_time], [xhist3[2,trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

savefig("Test05.svg")

In [ ]:
plot(thist,xhist3[5,:], label="y MPC")
plot!(thist, drefyhist3, label="cos")

xlabel!("time")
scatter!([land_time], [drefyhist3[trunc(Int, land_time/simulation.h_universe)]], label="Mark", color=:red, markersize=4)

savefig("Test06.svg")

In [ ]:
plot(thist[1:end-1], uhist3[1,:], label="u1 MPC")
xlabel!("Time")

savefig("Test07.svg")

In [ ]:
plot(thist[1:end-1], uhist3[2,:], label="u2 MPC")
xlabel!("Time")

savefig("Test08.svg")